## Particle Filter-based Visual Object Tracking (Shape Matching Game)

This notebook implements a visual tracking algorithm (using a particle filter) combined with an interactive shape matching game. The game uses a moving red object (held by the user and detected via background substraction, HSV filtering and morphological operations - pixel-level operations) to control a ball (in the form of a triangle, square, rectangle, or circle) whose position is tracked by a particle filter. In the game, the ball (displayed as a filled shape) must match the target shape (displayed as an outline) to score points, while avoiding obstacles.

 The project is implemented in Python using OpenCV for image processing and  (red) object detection, NumPy for numerical computations, and Pygame for sound effects.

This implementation was carried out as part of the Image and Video Analysis Course at UIB.

### Authors:
- Halidu Abdulai
- Ahmad Kamal Baig

`Disclaimer`: It is possible that you may not be able to run the game from Jupyter notebook due to your system settings blocking the pop-up window! In that case, you can run the demo by simply running the `main.py` file.

---

#### Table of Content
1. [Introduction](#1-introduction-to-the-real-world-problem)
    - [1.1 Potential Real-world application of our designed system](#11-potential-real-world-applications)
2. [Dectection and tracking algorithm (Particle Filter)](#2-detection-and-tracking-algorithm)
    - [2.1 Description of detection algorithm](#21-detection-algorithm---obtaining-measurement)
    - [2.2 Reasons for the choice of a particle filter](#22-why-use-a-particle-filter)
    - [2.3 The role of Particle Filter in the implementation](#23-how-it-works-in-our-implementation)
    - [2.4 Advantages of particle filter in the context of our chosen scenario](#24-advantages-of-using-a-particle-filter)
    - [2.5 How the Particle Filter is modoled in the designed system](#25-how-we-modeled-the-particle-filter-in-our-system)
3. [Dependencies installation](#3-dependencies-installation)
4. [Step-by-step Implementation Guidelines](#4-step-by-step-tutorial-implementation)
    - [4.1 Importing Libraries](#41-import-necessary-libraries)
    - [4.2 The Ball class (shapes that need to be matched to a target)](#42-the-ball-class)
    - [4.3 The Target class (outlines for which shapes [the ball] must be matched to)](#43-the-target-class)
    - [4.4 The Obstacles class (moving obstacles on the screen)](#44-the-obstacle-class)
    - [4.5 The PowerUp class (shield to protect the ball agaisnt obstacles)](#45-the-powerup-class-shield)
    - [4.6 The ParticleEffect class (Reaction effects) ](#46-particleeffect-class)
    - [4.7 Game Entry point](#47-game-class-main-controller)
5. [Running the Program](#5-game-entry-point---running-the-game)
6. [Challenges, Lessons Learned and Possible Improvements](#6-challenges-and-possible-improvements)

## 1. Introduction to the Real-World Problem
Accurate object tracking is a fundamental challenge in various real-world applications, including surveillance systems, traffic monitoring, and virtual reality (VR) in the gaming industry. The ability to reliably track a moving object, despite challenges such as noise, occlusions, and lighting variations, is essential for ensuring the effectiveness of these applications.

This project simulates an interactive scenario in which a user controls a virtual ball using hand movements. The system utilizes a particle filter to track the ball’s position, allowing it to navigate and interact with different elements within a dynamic environment, such as targets, obstacles, and power-ups. While this scenario is designed as a game, the underlying object tracking techniques are directly applicable to practical problems, making the project a valuable learning tool for real-world tracking applications.

#### 1.1 Potential Real-World Applications
Beyond its use in gaming, this system has significant potential in other domains, including healthcare rehabilitation and education:

- Rehabilitation & Physical Therapy: The tracking mechanism used in this project could be applied in rehabilitation programs, where medical professionals monitor a patient’s hand, finger, or arm movements after an injury or neurological disorder (e.g., stroke, traumatic brain injury). Introducing this system in the form of a game (Gamification) can serve as a motivational tool for patients, encouraging them to perform repetitive movements necessary for recovery.
- Education & Learning: The interactive nature of the game makes it well-suited for educational applications, particularly for young children learning shapes and spatial awareness. By matching different shapes through hand movements, children can develop cognitive and motor skills in an engaging and immersive way.


## 2. Detection and Tracking Algorithm

In our game, the ball's position is tracked using a [Particle Filter](https://en.wikipedia.org/wiki/Particle_filter), a probabilistic approach well-suited for visual tracking in uncertain or noisy environments. This algorithm is particularly effective in scenarios where traditional deterministic tracking methods struggle, such as when dealing with occlusions, lighting variations, and sensor noise.

#### 2.1 Detection Algorithm - Obtaining Measurement:
In our implementation, the measurement is obtained by detecting a moving red object within each video frame by combining background subtraction, color segmentation, and morphological opeartions on detected contours. This method isolates dynamic red regions while filtering out static red elements.

##### 2.1.1. Background Subtraction:
In this step, we apply a background subtractor (`cv2.createBackgroundSubtractorMOG2`) to each input frame to generate a `foreground mask`. This mask highlights moving objects, helping to ignore static regions in the background.

##### 2.1.2. Color Conversion and Red Masking:
  We convert RGB color space to HSV and use the two color ranges to capture the complete spectrum of red color:
  
  $$
  \begin{align*}
  \text{Range 1: } & [0, 120, 70] \quad \text{to} \quad [10, 255, 255] \\\\
  \text{Range 2: } & [170, 120, 70] \quad \text{to} \quad [180, 255, 255]
  \end{align*}
  $$

##### 2.1.3. Masking:
- **Mask Combination:**  
  Two masks are created—one for each red range—and then combined to cover all possible red color ranges. The red mask is compared to the foreground mask using a bitwise AND operation to find the region of interest (ROI). We do this to ensure that only the moving red regions are considered.

- **Mask Cleaning:**  
  The combined mask is refined using:
  - **Erosion:** Removes small noise.
  - **Dilation:** Restores the eroded parts.
  - **Gaussian Blur:** Smooths the mask to further reduce artifacts.

##### 2.1.4. Contour Detection:
- **Contour Extraction:**  
  We extract the largest contour of the mask from the previous step and use that as the contour of the moving red object

- **Centroid Calculation:**  
  The centroid of the largest contour is computed using image moments:

  $$
  c_x = \frac{M_{10}}{M_{00}}, \quad c_y = \frac{M_{01}}{M_{00}}
  $$

##### 2.1.5. Tip Detection Using Convex Hull:
- **Convex Hull:**  
  A convex hull is generated around the selected contour to encapsulate its shape.

- **Farthest Point Determination:**  
  The algorithm then identifies the point on the hull that is farthest from the centroid. This point is assumed to be the tip of the red object.

##### 2.1.6. Output:
- **Measurement Point:**  
  If a valid red object is detected, the coordinates of the farthest point (i.e., the tip) are returned as the measurement. Otherwise, the function returns `None`.

##### **Summarizing the above steps, the detection algorithm consists of:**

1. **Extract Foreground:** Apply background subtraction to isolate moving objects.
2. **Segment Red Regions:** Convert the frame to HSV, define red color ranges, and create a combined red mask.
3. **Refine Mask:** Clean the mask using erosion, dilation, and Gaussian blur.
4. **Detect Contours:** Identify contours and select the largest one 
5. **Compute Centroid:** Calculate the centroid of the selected contour.
6. **Identify Tip:** Use the convex hull to find the point farthest from the centroid.
7. **Return Measurement:** Output the tip’s coordinates as the detected red object’s position.


#### 2.2 Tracking Algorithm: Why Use a Particle Filter?
A particle filter models the object's state (in this case, the ball’s position and velocity) as a set of particles, each representing a possible location of the object. By continuously predicting and updating these particles based on motion dynamics and sensor observations, the algorithm can effectively estimate the object's true position.

#### 2.3 How It Works in Our Implementation
<b>Prediction (Motion Model)</b>: Each particle represents a possible state (position, velocity) of the ball.
Particles are propagated according to estimated motion dynamics, with some random noise added to account for uncertainty.

<b>Update (Correction with red object Tracking Data)</b>: When a measurement is available (e.g., the detected red object position processed via background subtraction, HSV filtering and morphological operations), the algorithm compares the predicted particle positions with the observed measurement.
Particles closer to the measurement receive higher weights, while those farther away receive lower weights.
A resampling process is then applied, selecting particles proportionally to their weights, ensuring the most likely positions of the ball are retained.

#### 2.4 Advantages of Using a Particle Filter

The decision to use a particle filter over other well-known filters, such as the Kalman filter, is driven by several key advantages. The particle filter is particularly well-suited for our application because it can effectively model non-linear and non-Gaussian systems. Additionally, it offers the following benefits:

- <b>Robustness to Noisy Measurements</b>: This is especially valuable when tracking objects detected through pixel operations, which can introduce fluctuations and inaccuracies.
- <b>Resilience to Occlusion</b>: Unlike traditional methods, the particle filter algorithm can estimate the ball's location even when it is temporarily occluded.
- <b>Adaptability to Varying Lighting Conditions</b>: Since the tracking relies on a probabilistic model rather than strict visual cues, it can dynamically adjust to different environments.

#### 2.5 How We Modeled the Particle Filter in Our System

The particle filter is implemented by generating multiple particles (samples) that represent possible states of the ball. The filter then iteratively performs prediction and correction steps to estimate the ball's position based on noisy measurements (from detection a red object. Since the user will appear in the frame, it makes sense to find something unique to detect, hence the decision to detect a red object. We expect the user to hold a red object in their hand which will serve as the region of interest, ROI).

##### 2.5.1. Particle Initialization:
- Each particle represents a possible state of the ball, defined by:
  
$$
\text{State Vector: } \mathbf{x} = [x, y, v_x, v_y]
$$

- **Position ($x, y$):** Randomly initialized around the ball’s initial position with Gaussian noise.  
- **Velocity ($v_x, v_y$):** Randomly initialized with small values and Gaussian noise.

##### 2.5.2. Prediction (Motion Model): 
In the prediction step, we update each particle’s position and velocity based on the motion model with added noise:

- **Position Update:**  
$$
x = x + v_x + \epsilon_{x}, \quad y = y + v_y + \epsilon_{y}
$$

- **Velocity Update:**  
$$
v_x = v_x + \eta_{x}, \quad v_y = v_y + \eta_{y}
$$

Where:  
- $\epsilon_{x}, \epsilon_{y} \sim \mathcal{N}(0, \sigma^2_{\text{motion}})$ (Motion noise)  
- $\eta_{x}, \eta_{y} \sim \mathcal{N}(0, \sigma^2_{\text{accel}})$ (Acceleration noise)

Additionally:
- **Boundary Handling:** When particles hit the screen boundaries, their velocities are reversed to simulate a bounce:

$$
\begin{cases}
v_x = -v_x, & \text{if } x < 0 \text{ or } x > \text{width} \\[12pt]
v_y = -v_y, & \text{if } y < 0 \text{ or } y > \text{height}
\end{cases}
$$

##### 2.5.3. Update (Correction Step): 
In the update step, we incorporate the measurement (red object position) and assign weights to the particles based on their distance from the measurement.  

- **Noisy Measurement:**  
  The measurement ($x_m, y_m$) is obtained from detecting a red object held by the user. This measurement is assumed to be noisy since we use pixel-level operations to obtain the position of the object of interest:

$$
x_m = x_{red\_obj}, \quad y_m = y_{red\_obj} 
$$

##### 2.5.4. Weight Calculation: 
We calculate each particle's weight based on its distance from the noisy measurement using a Gaussian likelihood:

- **Distance to Measurement:**  
$$
d_i = \sqrt{(x_i - x_m)^2 + (y_i - y_m)^2}
$$

- **Particle Weight (Gaussian Likelihood):**  
$$
w_i = \frac{1}{\sqrt{2\pi}\sigma} \exp\left(-\frac{d_i^2}{2\sigma^2}\right)
$$

Where:  
- $\sigma$ is the standard deviation of the measurement model.
 

- **Weight Normalization:**  
The weights are normalized to sum to 1:

$$
w_i = \frac{w_i}{\sum_{j=1}^{N} w_j}
$$

##### 2.5.5. Number of Effective Particles (Neff) and Resampling:
To reduce particle degeneracy, we compute the **Number of Effective Particles (Neff):**

- **Number of Effective Particles (Neff):**  
$$
\text{Neff} = \frac{1}{\sum_{i=1}^{N} w_i^2}
$$

- **Resampling Condition:**  
If **Neff** is below half the number of particles (resampling threshold):  
$$
\text{If } \text{Neff} < \frac{N}{2}, \quad \text{then Resample}
$$

##### 2.5.6. Resampling: 
We resample the particles with replacement using their weights as probabilities. Particles closer to the measurement have a higher chance of being selected. 

After resampling, a small amount of noise is added to the velocities to maintain particle diversity:

$$
v_{x,i} = v_{x,i} + \eta_{x}, \quad v_{y,i} = v_{y,i} + \eta_{y}
$$

##### 2.5.7. Position Estimation: 
The estimated position of the ball is computed as the mean of the particle positions:

$$
\hat{x} = \frac{1}{N} \sum_{i=1}^{N} x_i, \quad \hat{y} = \frac{1}{N} \sum_{i=1}^{N} y_i
$$

##### **In a few words, the Particle Filter Algorithm consists of the following steps:**

1. **Initialize:** Particles around the ball’s initial position with random velocities.  
2. **Predict:** Estimate particles' states based on their positions, velocities and motion noise.  
3. **Update:** Assign weights based on the likelihood of measurement.  
4. **Check Neff:** If below threshold, resample particles.  
5. **Estimate:** Use the average of particle positions as the ball’s position.  
6. **Repeat:** Continuously iterate for each frame.  

With this implementation, the particle filter effectively tracks the ball using noisy `red object` measurements, even in the presence of occlusions or sudden movements, making the gameplay responsive and challenging.

[Table of content](#table-of-content)


## 3. Dependencies Installation

Before running the code, ensure that you have the following packages installed:

- **OpenCV:** For computer vision tasks and drawing shapes
- **Pygame:** For sound effects
- **NumPy:** For numerical operations

You can install them using the following command (uncomment and run the cell below if needed):

In [31]:
# Uncomment the following lines to install required packages
# !pip install opencv-python  pygame numpy

## 4. Step-by-Step Tutorial: Implementation
The logic of our developed systems is summarized by the flowchart below

[![](https://mermaid.ink/img/pako:eNqdVltP4zgU_itHkXYE2oA6pS1Qzc4otAWqoRcIl4HtPriJ23pJ48hxWjqI_z7HlziB3X1ZpIp-9vE537n31Yt4TL2ut0j4NloRIeG2P0sB_4I_Z14o1ckFWdOZ95c5PsPjYcokIwn7SfUdTOZ_00jmX-bi694ZSRIfbolYUpn7eJVLEiUUv075loqDIst9JThFzQwvYLBYqMc-3A19CIqY8X1nrOc43LOYcuiRTBaiItPH-xFhqaFxxXnmrgZ4ZcXhXNQ9OMeb84Rl5hj2RkwILiqjF-olTzcUzRoRyeEyvHcClyjQpxJZww2NrffW-eh5KXiRxhAWcylIJBlP4Xf1HM5ZIqlg6bIy9R01TQWNmZELJc26StFdFhNJwcUIYyCptvDiw86HDf7b7PZhy-QKRlw__qTugyiiCRVEn4w5yyu3h68YK0pyDMiaphKCDWEJmSf028x7MyJX2nMhqKajmfT4OiuQygNly5XMYU5ydBmV11QpOYIuDzYkKRRvk1K2QeJknSn-WCqOyAiJ_KsEJnBbkRkjmSDGDCqpG5orOQyejoKFVYRyYAsYUxrTuIruBDXYSLoyxGhtNGknNa2kwhWjSQy3bE0F7KHGQHOsNF6rAK1o9Aw9niQsV1HWOXC17URvFH31vKZYBcmUszGBFrQeZO3ehf9lwrngRG9VKy4qOZ0wrMgionBJ0UyunUi5tb_vG4GcSny0nnPsU8GWS2TSJ2uyLBPnDNw5LqhGNTYMUizh3LY3PPG0Sus9pnWYG7FwRTIKBEZERqsqpQ8lYV1imu4wjQRWEoYo4oLiEDCZMPSUQMkwLLC089xSxGmSkJ15BKFqOMfjhzGCimtm3kXlnd6Bav73Wqc0JYncfdD7iHonUURUpNFHvM7INnWZ13VJcWjsoN1owEINDhxq7cZvgHM1jWpV9Nh0cV2g7St8laDjmujewKhAvSyV-b6uGc1Ki6Ghj7SOdJ9g973gvDEeqnSZ0IwYFozENGEHkWhF4xrNI8iMDafrqWqFT9AXZAsfp7QTDdR60CL_ZwNUJoOzUs_dEPZsEdg0GZf9slZtE6lZWNQ19LDwDAl065umYKv_yx_Q2HfVF6htEa44Ug5GA5jcD24wSoLS1A0w_PxzkwRqlagTUONdLYNyr4zpi_ywXAK1XXoJJSlmCluNFwLL1g6tRFe6fe1DH1Mj-A4eWBrzLfp7XTBZX4FWJRwcfIUzu3416Nn1qIHd130NBnb7aXBuF54GF3a7aXBpN5kG3-0y0mBotwUCeMR5qc6uamdjro8mdmNoMLKDvf5mXDt7_2ZcBxMNpnYUa3BtZ60GNxUo1YR2wNZBqMGtnYsa3NkZpsG9nVF1hg-1M6v6h51TGjw27UB5hx6bBh6V7WegRRo82W7SIAjKhjHwrKx7A3tlEdeZBf36qeUWDMpyrCc6MGkPMNWe7-FSWRMW4--5V3U98-SKquLs4teYiOeZN0vfUI4Ukoe7NPK6UhTU9_Any3LldRckyREVegT0GVliabvTjKRPnK_LJwi97qv34nU_n3QO281W57h50j5utRutY9_b4fHR8eFp4_Ppaat9ctL6fNpuvfneT62hcdg56jRPm632cfuk0e50Wm-_ALQUNB0?type=png)](https://mermaid.live/edit#pako:eNqdVltP4zgU_itHkXYE2oA6pS1Qzc4otAWqoRcIl4HtPriJ23pJ48hxWjqI_z7HlziB3X1ZpIp-9vE537n31Yt4TL2ut0j4NloRIeG2P0sB_4I_Z14o1ckFWdOZ95c5PsPjYcokIwn7SfUdTOZ_00jmX-bi694ZSRIfbolYUpn7eJVLEiUUv075loqDIst9JThFzQwvYLBYqMc-3A19CIqY8X1nrOc43LOYcuiRTBaiItPH-xFhqaFxxXnmrgZ4ZcXhXNQ9OMeb84Rl5hj2RkwILiqjF-olTzcUzRoRyeEyvHcClyjQpxJZww2NrffW-eh5KXiRxhAWcylIJBlP4Xf1HM5ZIqlg6bIy9R01TQWNmZELJc26StFdFhNJwcUIYyCptvDiw86HDf7b7PZhy-QKRlw__qTugyiiCRVEn4w5yyu3h68YK0pyDMiaphKCDWEJmSf028x7MyJX2nMhqKajmfT4OiuQygNly5XMYU5ydBmV11QpOYIuDzYkKRRvk1K2QeJknSn-WCqOyAiJ_KsEJnBbkRkjmSDGDCqpG5orOQyejoKFVYRyYAsYUxrTuIruBDXYSLoyxGhtNGknNa2kwhWjSQy3bE0F7KHGQHOsNF6rAK1o9Aw9niQsV1HWOXC17URvFH31vKZYBcmUszGBFrQeZO3ehf9lwrngRG9VKy4qOZ0wrMgionBJ0UyunUi5tb_vG4GcSny0nnPsU8GWS2TSJ2uyLBPnDNw5LqhGNTYMUizh3LY3PPG0Sus9pnWYG7FwRTIKBEZERqsqpQ8lYV1imu4wjQRWEoYo4oLiEDCZMPSUQMkwLLC089xSxGmSkJ15BKFqOMfjhzGCimtm3kXlnd6Bav73Wqc0JYncfdD7iHonUURUpNFHvM7INnWZ13VJcWjsoN1owEINDhxq7cZvgHM1jWpV9Nh0cV2g7St8laDjmujewKhAvSyV-b6uGc1Ki6Ghj7SOdJ9g973gvDEeqnSZ0IwYFozENGEHkWhF4xrNI8iMDafrqWqFT9AXZAsfp7QTDdR60CL_ZwNUJoOzUs_dEPZsEdg0GZf9slZtE6lZWNQ19LDwDAl065umYKv_yx_Q2HfVF6htEa44Ug5GA5jcD24wSoLS1A0w_PxzkwRqlagTUONdLYNyr4zpi_ywXAK1XXoJJSlmCluNFwLL1g6tRFe6fe1DH1Mj-A4eWBrzLfp7XTBZX4FWJRwcfIUzu3416Nn1qIHd130NBnb7aXBuF54GF3a7aXBpN5kG3-0y0mBotwUCeMR5qc6uamdjro8mdmNoMLKDvf5mXDt7_2ZcBxMNpnYUa3BtZ60GNxUo1YR2wNZBqMGtnYsa3NkZpsG9nVF1hg-1M6v6h51TGjw27UB5hx6bBh6V7WegRRo82W7SIAjKhjHwrKx7A3tlEdeZBf36qeUWDMpyrCc6MGkPMNWe7-FSWRMW4--5V3U98-SKquLs4teYiOeZN0vfUI4Ukoe7NPK6UhTU9_Any3LldRckyREVegT0GVliabvTjKRPnK_LJwi97qv34nU_n3QO281W57h50j5utRutY9_b4fHR8eFp4_Ppaat9ctL6fNpuvfneT62hcdg56jRPm632cfuk0e50Wm-_ALQUNB0)

Below are the code cells that implement the game. Each section of the code is documented to explain its purpose and functionality.

#### 4.1 Import Necessary libraries

[Click here to return back to the table of content](#table-of-content)

In [32]:
import cv2
import numpy as np
import random
import pygame
import logging


#### 4.2 The Ball Class
The Ball class represents a game object that is tracked using a particle filter. It is responsible for initializing the ball’s position, maintaining a set of particles representing possible states (position and velocity), estimating their states based on motion dynamics (including abrupt movements), and rendering the ball within a visual frame. The class integrates probabilistic tracking and adaptive resampling to ensure accurate position estimation even during rapid movements or temporary occlusions.

The Key Functionalities include:

1. **Initialization (__init__ method)**
    - Initializes the canvas size, ball parameters, and shape selection (randomly chosen from predefined shapes such as circle, square, rectangle, or triangle).
    - Calls `_init_position()` to ensure the ball starts fully within the canvas boundaries.
    - Calls `_init_particles()` to generate an initial distribution of particles around the ball’s position, with each particle represented by a 4D state vector \([x, y, v_x, v_y]\).

2. **Ball Position Initialization (_init_position)**
    - Determines a random starting position based on the ball's shape and dimensions.
    - Ensures the ball does not exceed the canvas boundaries.

3. **Particle Initialization (_init_particles)**
    - Creates a set of particles, each represented as a 4D state \([x, y, v_x, v_y]\).
    - Initializes the particles’ positions around the initial ball position using a normal distribution.
    - Assigns each particle a small random velocity, allowing the filter to model the ball's dynamics.
    - These particles serve as hypotheses for the ball’s state (both position and velocity) in subsequent frames.

4. **Particle Filter Update (update method)**
    - **Prediction Step:**
        - Estimates each particle’s position using its current velocity, with additional motion noise.
        - Estimates each particle’s velocity by adding acceleration noise to account for potential abrupt movements.
    - **Correction Step (if a measurement is available):**
        - Reweights particles based on the proximity of their positions to the observed measurement (i.e., a moving red object is detected) using a Gaussian likelihood.
        - Computes the effective sample size (a measure of particle degeneracy).
        - **Adaptive Resampling:** Resamples particles based on their weights only if the effective sample size falls below a set threshold, thereby maintaining particle diversity while favoring particles closer to the measurement.
    - Ensures the particles remain within the canvas boundaries.

5. **Position Estimation (get_position method)**
    - Computes the mean of the \([x, y]\) positions of all particles to estimate the most probable ball location.
    - Returns a smoothed position that reduces the influence of outlier particles.

6. **Rendering the Ball (draw method)**
    - Uses OpenCV drawing functions to render the ball based on its shape.
    - Visualizes the particles (for debugging or demonstration purposes), displaying their current positions.
    - Supports rendering for various geometric shapes (circle, square, rectangle, triangle).

In a few words, this class works as follows:
- **Initialization:** A set of particles (with positions and velocities) is created around the initial ball position.
- **Prediction:** Each particle’s state is estimated using its position, velocity and random noise, simulating realistic motion dynamics.
- **Update (Correction):** If a measurement (moving red object detected) is available:
    - Particles are weighted based on their proximity to the measurement.
    - Adaptive resampling is performed if the effective sample size is low, favoring particles closer to the measured position while preserving diversity.
- **Position Estimation:** The mean of the particles’ positions is computed and returned as the ball’s estimated location.

The implementation is shown in the cell below!


In [33]:
class Ball:
    """
    Represents the game ball that uses a Particle filter to track its position.
    The state vector for each particle is [x, y, vx, vy].
    """
    def __init__(self, canvas_size, ball_params, shapes, num_particles=300):
        self.canvas_size = canvas_size          # (height, width, channels)
        self.ball_params = ball_params          # Dictionary containing shape parameters
        self.shapes = shapes                    # List of possible shapes
        self.num_particles = num_particles
        self.shape = random.choice(shapes)
        self._init_position()
        self._init_particles()

    def _init_position(self):
        if self.shape == "circle":
            r = self.ball_params["circle"]["radius"]
            x = random.randint(r, self.canvas_size[1] - r)
            y = random.randint(r, self.canvas_size[0] - r)
        elif self.shape == "square":
            s = self.ball_params["square"]["side"]
            x = random.randint(s // 2, self.canvas_size[1] - s // 2)
            y = random.randint(s // 2, self.canvas_size[0] - s // 2)
        elif self.shape == "rectangle":
            w = self.ball_params["rectangle"]["width"]
            h = self.ball_params["rectangle"]["height"]
            x = random.randint(w // 2, self.canvas_size[1] - w // 2)
            y = random.randint(h // 2, self.canvas_size[0] - h // 2)
        elif self.shape == "triangle":
            s = self.ball_params["triangle"]["side"]
            x = random.randint(s, self.canvas_size[1] - s)
            y = random.randint(s, self.canvas_size[0] - s)
        self.position = np.array([x, y])

    def _init_particles(self):
        # Each particle state: [x, y, vx, vy]
        self.particles = np.zeros((self.num_particles, 4))
        # Initialize positions around the ball's initial position with some noise
        self.particles[:, :2] = np.random.randn(self.num_particles, 2) * 20 + self.position
        # Initialize velocities with small random values
        self.particles[:, 2:] = np.random.randn(self.num_particles, 2) * 5

    def update(self, measurement=None, motion_noise=5, accel_noise=2):
        """
        Updates the particle filter with a dynamic state model.
        - Prediction: Each particle's position is updated using its velocity,
          plus added motion noise. The velocity is also perturbed with acceleration noise.
        - Correction: When a measurement is available, particle weights are computed
          based on the distance from the measurement. Adaptive resampling is then performed
          based on the effective sample size.
        """
        # Prediction Step (State estimation)
        # Update position: x = x + vx, y = y + vy plus motion noise.
        # Formula derived from x = x_0 + vt + 0.5at^2, where assuming t = 1 and modeling acceleration term as noise
        self.particles[:, :2] += self.particles[:, 2:] + np.random.randn(self.num_particles, 2) * motion_noise
        # Update velocity with some random acceleration noise (v = v_0 + at, assuming t = 1). 
        self.particles[:, 2:] += np.random.randn(self.num_particles, 2) * accel_noise

        # Keep particle positions within canvas boundaries and bounce off boundaries
        # Check left and right boundaries
        left_collision = self.particles[:, 0] < 0
        right_collision = self.particles[:, 0] > self.canvas_size[1] - 1
        self.particles[left_collision, 0] = 0
        self.particles[right_collision, 0] = self.canvas_size[1] - 1
        self.particles[left_collision | right_collision, 2] *= -1  # Reverse x-velocity

        # Check top and bottom boundaries
        top_collision = self.particles[:, 1] < 0
        bottom_collision = self.particles[:, 1] > self.canvas_size[0] - 1
        self.particles[top_collision, 1] = 0
        self.particles[bottom_collision, 1] = self.canvas_size[0] - 1
        self.particles[top_collision | bottom_collision, 3] *= -1  # Reverse y-velocity

        # Correction (Update) Step 
        if measurement is not None:
            # Compute Euclidean distances between each particle's (x,y) and the measurement.
            distances = np.linalg.norm(self.particles[:, :2] - measurement, axis=1)
            # Compute weights using a Gaussian likelihood.
            sigma = 50.0
            weights = np.exp(- (distances**2) / (2 * sigma**2))
            weight_sum = np.sum(weights)
            if weight_sum > 0:
                weights /= weight_sum
            else:
                weights = np.ones(self.num_particles) / self.num_particles

            # Compute effective sample size (Neff).
            effective_N = 1.0 / np.sum(weights**2)
            # Resample only if effective sample size falls below threshold.
            threshold = self.num_particles / 2.0
            if effective_N < threshold:
                indices = np.random.choice(self.num_particles, size=self.num_particles, p=weights)
                self.particles = self.particles[indices]
                # Optionally, add a small noise to velocities after resampling
                self.particles[:, 2:] += np.random.randn(self.num_particles, 2) * accel_noise

    def get_position(self):
        """
        Returns the estimated ball position as the mean of the (x,y) positions of the particles.
        """
        return np.mean(self.particles[:, :2], axis=0)

    def draw(self, frame):
        """
        Draws the ball (using the current estimated position) on the frame.
        The appropriate drawing method is chosen based on the ball's shape.
        """
        pos = self.get_position().astype(int)
        color = (225, 105, 65)  # Ball color (BGR)
        if self.shape == "circle":
            r = self.ball_params["circle"]["radius"]
            cv2.circle(frame, (pos[0], pos[1]), r, color, -1, cv2.LINE_AA)
        elif self.shape == "square":
            s = self.ball_params["square"]["side"]
            top_left = (pos[0] - s // 2, pos[1] - s // 2)
            bottom_right = (pos[0] + s // 2, pos[1] + s // 2)
            cv2.rectangle(frame, top_left, bottom_right, color, -1)
        elif self.shape == "rectangle":
            w = self.ball_params["rectangle"]["width"]
            h = self.ball_params["rectangle"]["height"]
            top_left = (pos[0] - w // 2, pos[1] - h // 2)
            bottom_right = (pos[0] + w // 2, pos[1] + h // 2)
            cv2.rectangle(frame, top_left, bottom_right, color, -1)
        elif self.shape == "triangle":
            s = self.ball_params["triangle"]["side"]
            pt1 = (pos[0], pos[1] - int(s / np.sqrt(3)))
            pt2 = (pos[0] - s // 2, pos[1] + int(s / (2 * np.sqrt(3))))
            pt3 = (pos[0] + s // 2, pos[1] + int(s / (2 * np.sqrt(3))))
            pts = np.array([pt1, pt2, pt3], np.int32)
            cv2.fillPoly(frame, [pts], color)
        # (Optional) Draw particles for debugging/visualization.
        for p in self.particles:
            cv2.circle(frame, (int(p[0]), int(p[1])), 2, (100, 0, 0), -1)

#### 4.3 The Target Class
The Target class represents a target zone within the game where the player must guide the ball to score points. The target is a geometric shape (i.e., circle, square, rectangle, triangle) positioned at specific locations on the canvas. If the ball enters the target area and its shape matches the target’s shape, the player scores points.

Key Functionalities of this Class include:
1. Initialization (__init__ method)
    - Assigns the shape, canvas size, and target parameters (size properties for each shape).
    - The position is initially set to None and is later assigned externally.

2. Margin Calculation (get_margin method - Static)
    - Determines the required margin around the target based on its shape.
    - This ensures the target does not exceed the canvas boundaries.

    For example:
    - Circles require a radius margin.
    - Squares require a half-side margin.
    - Rectangles use the largest dimension divided by two.
    - Triangles require a side-length margin.

3. Target Creation (create_targets method - Static)
    - Randomly assigns four targets (one in each corner of the canvas) using different shapes.
    - Ensures that each target has a unique shape by shuffling the shape list.
    - Determines the position of each target while maintaining proper spacing from the edges.

4. Drawing the Target (draw method)
    - Uses OpenCV to render the target as an outlined shape (not filled).

    The function:
    - Draws a circle, square, rectangle, or triangle at the assigned position.
    - Uses polylines for triangles and rectangles/circles for other shapes.
    - Sets the outline thickness to differentiate targets from the ball.

5. Checking if the Ball is Inside the Target (point_inside method)
    - Determines if a given point (ball position) is within the target's boundaries.

    The logic differs for each shape:
    - Circle: Checks if the ball’s center is within the target radius.
    - Square/Rectangle: Ensures the ball’s center falls within the bounding box.
    - Triangle: Uses OpenCV's point-in-polygon test to verify containment.

##### How the Target Class Integrates with the Game
- Each game round places four targets at different corners.
- The player must navigate the ball to the correct target based on its shape.
- If the ball enters a matching target, the player scores points.
- If the ball enters an incorrect target, the player loses a life.

The implementation is shown in the cell below

[Click here to return to the table of content](#table-of-content)

In [34]:
class Target:
    """
    Represents a target zone on the canvas. The target is shown as an outline of a shape.
    When the ball enters the target and the shapes match, points are scored.
    """
    def __init__(self, shape, canvas_size, target_params):
        self.shape = shape
        self.canvas_size = canvas_size
        self.target_params = target_params
        self.position = None  # To be set externally

    @staticmethod
    def get_margin(shape, target_params):
        """
        Returns a margin based on the target's shape.
        """
        if shape == "circle":
            return target_params["circle"]["radius"]
        elif shape == "square":
            return target_params["square"]["side"] // 2
        elif shape == "rectangle":
            return max(target_params["rectangle"]["width"], target_params["rectangle"]["height"]) // 2
        elif shape == "triangle":
            return target_params["triangle"]["side"]
        return 50

    @staticmethod
    def create_targets(canvas_size, target_params, shapes):
        """
        Creates target zones at the four corners of the canvas.
        Each target is given a unique shape (shuffled order).
        """
        targets = []
        shapes_copy = shapes.copy()
        random.shuffle(shapes_copy)
        corners = ["top_left", "top_right", "bottom_left", "bottom_right"]
        for corner in corners:
            shape = shapes_copy.pop()
            margin = Target.get_margin(shape, target_params)
            if corner == "top_left":
                x = random.randint(margin, margin + 50)
                y = random.randint(margin, margin + 50)
            elif corner == "top_right":
                x = random.randint(canvas_size[1] - margin - 50, canvas_size[1] - margin)
                y = random.randint(margin, margin + 50)
            elif corner == "bottom_left":
                x = random.randint(margin, margin + 50)
                y = random.randint(canvas_size[0] - margin - 50, canvas_size[0] - margin)
            elif corner == "bottom_right":
                x = random.randint(canvas_size[1] - margin - 50, canvas_size[1] - margin)
                y = random.randint(canvas_size[0] - margin - 50, canvas_size[0] - margin)
            target = Target(shape, canvas_size, target_params)
            target.position = np.array([x, y])
            targets.append(target)
        return targets

    def draw(self, frame):
        """
        Draws the target outline on the canvas.
        """
        pos = self.position.astype(int)
        color = (225, 105, 65)  # Outline color
        thickness = 3
        if self.shape == "circle":
            r = self.target_params["circle"]["radius"]
            cv2.circle(frame, (pos[0], pos[1]), r, color, thickness, cv2.LINE_AA)
        elif self.shape == "square":
            s = self.target_params["square"]["side"]
            top_left = (pos[0] - s // 2, pos[1] - s // 2)
            bottom_right = (pos[0] + s // 2, pos[1] + s // 2)
            cv2.rectangle(frame, top_left, bottom_right, color, thickness)
        elif self.shape == "rectangle":
            w = self.target_params["rectangle"]["width"]
            h = self.target_params["rectangle"]["height"]
            top_left = (pos[0] - w // 2, pos[1] - h // 2)
            bottom_right = (pos[0] + w // 2, pos[1] + h // 2)
            cv2.rectangle(frame, top_left, bottom_right, color, thickness)
        elif self.shape == "triangle":
            s = self.target_params["triangle"]["side"]
            pt1 = (pos[0], pos[1] - int(s / np.sqrt(3)))
            pt2 = (pos[0] - s // 2, pos[1] + int(s / (2 * np.sqrt(3))))
            pt3 = (pos[0] + s // 2, pos[1] + int(s / (2 * np.sqrt(3))))
            pts = np.array([pt1, pt2, pt3], np.int32)
            cv2.polylines(frame, [pts], isClosed=True, color=color, thickness=thickness)

    def point_inside(self, point):
        """
        Checks whether the given point lies inside the target's boundary.
        """
        pos = self.position
        if self.shape == "circle":
            r = self.target_params["circle"]["radius"]
            return np.linalg.norm(point - pos) < r
        elif self.shape == "square":
            s = self.target_params["square"]["side"]
            top_left = pos - np.array([s / 2, s / 2])
            bottom_right = pos + np.array([s / 2, s / 2])
            return (top_left[0] <= point[0] <= bottom_right[0]) and (top_left[1] <= point[1] <= bottom_right[1])
        elif self.shape == "rectangle":
            w = self.target_params["rectangle"]["width"]
            h = self.target_params["rectangle"]["height"]
            top_left = pos - np.array([w / 2, h / 2])
            bottom_right = pos + np.array([w / 2, h / 2])
            return (top_left[0] <= point[0] <= bottom_right[0]) and (top_left[1] <= point[1] <= bottom_right[1])
        elif self.shape == "triangle":
            s = self.target_params["triangle"]["side"]
            pt1 = (pos[0], pos[1] - int(s / np.sqrt(3)))
            pt2 = (pos[0] - s // 2, pos[1] + int(s / (2 * np.sqrt(3))))
            pt3 = (pos[0] + s // 2, pos[1] + int(s / (2 * np.sqrt(3))))
            pts = np.array([pt1, pt2, pt3], np.int32)
            return cv2.pointPolygonTest(pts, (int(point[0]), int(point[1])), False) >= 0
        return False

#### 4.4 The Obstacle Class
The Obstacle class defines moving obstacles within the game environment. These obstacles are rectangular objects that move dynamically across the canvas, bouncing off the boundaries. Their speed increases as the game progresses, making the game more challenging.

Key Functionalities include:

1. Initialization (__init__ method)
    - Defines the canvas size and assigns a difficulty level (level).
    - Randomly selects the size of the obstacle (between 50×50 and 100×100 pixels).
    - Initializes the position within the canvas, ensuring the obstacle fits within the screen boundaries.
    - Assigns a random velocity to determine the movement direction and speed.
    - Speed scaling: The velocity increases as the game level progresses, making higher levels more difficult.

2. Updating the Obstacle's Position (update method)
    - Moves the obstacle based on its velocity vector.
    - Ensures that if the obstacle hits a canvas boundary, it bounces back by inverting its velocity along the respective axis.
    - This movement mechanism simulates realistic motion while keeping obstacles within the game area.

3. Rendering the Obstacle (draw method)
    - Uses OpenCV to draw the obstacle as a solid red rectangle.
    - The position is determined dynamically, based on the current frame.

4. Collision Detection with the Ball (collides_with method)
    - Checks if the ball enters the obstacle's rectangular area.
    - Uses bounding box overlap detection to determine if the ball’s center is within the obstacle’s dimensions.

    If a collision occurs:
    - The player loses a life (unless a shield power-up is active).
    - The game may reset the ball’s position and spawn new obstacles.
##### How the Obstacle Class Contributes to Gameplay
- The obstacles add difficulty by obstructing the player’s ability to guide the ball freely.
- Their dynamic movement creates an unpredictable challenge for the player.
- The progressive speed increase with higher levels ensures that gameplay becomes progressively harder.

In [35]:
class Obstacle:
    """
    Represents a moving obstacle (a rectangle) on the canvas. Its velocity increases with the game level.
    """
    def __init__(self, canvas_size, level):
        self.canvas_size = canvas_size
        self.level = level
        self.size = (random.randint(50, 100), random.randint(50, 100))
        self.position = np.array([
            random.randint(0, canvas_size[1] - self.size[0]),
            random.randint(0, canvas_size[0] - self.size[1])
        ], dtype=float)
        speed_factor = 1 + (level - 1) * 0.2
        self.velocity = np.array([
            random.choice([-5, -3, 3, 5]) * speed_factor,
            random.choice([-5, -3, 3, 5]) * speed_factor
        ], dtype=float)

    def update(self):
        """
        Updates the obstacle's position and bounces off canvas boundaries.
        """
        self.position += self.velocity
        if self.position[0] < 0 or self.position[0] + self.size[0] > self.canvas_size[1]:
            self.velocity[0] = -self.velocity[0]
        if self.position[1] < 0 or self.position[1] + self.size[1] > self.canvas_size[0]:
            self.velocity[1] = -self.velocity[1]

    def draw(self, frame):
        """
        Draws the obstacle as a filled red rectangle on the frame.
        """
        top_left = (int(self.position[0]), int(self.position[1]))
        bottom_right = (int(self.position[0] + self.size[0]), int(self.position[1] + self.size[1]))
        cv2.rectangle(frame, top_left, bottom_right, (0, 0, 255), -1)

    def collides_with(self, ball):
        """
        Checks if the obstacle collides with the ball.
        """
        ball_pos = ball.get_position()
        x, y = self.position
        w, h = self.size
        return (x <= ball_pos[0] <= x + w) and (y <= ball_pos[1] <= y + h)

#### 4.5 The PowerUp Class (Shield)
The PowerUp class introduces special game elements that grant players temporary advantages. In this implementation, the power-up provides a shield that protects the player from losing lives when colliding with obstacles or mismatched targets.

Key Functionalities Include:

1. Initialization (__init__ method)
    - Defines the canvas size to ensure the power-up is placed within the game boundaries.
    - Sets a radius of 30 pixels, determining the size of the power-up.
    - Randomly assigns a position within the canvas, ensuring it does not overlap the edges.
    - Defines the power-up type as "shield", which protects the player from damage.
    - Sets a duration of 300 frames, indicating how long the power-up effect lasts once collected.

2. Rendering the Power-Up (draw method)
    - Uses OpenCV to draw the power-up as a solid yellow circle at its assigned position. The decision for a yellow color is arbitrary. We may actually change it by the end of our final touches
    - This ensures clear visual feedback for the player.

##### How the PowerUp Class Contributes to Gameplay
- Power-ups appear randomly and provide temporary protection when collected.
- The shield effect prevents life loss when the ball hits obstacles or the wrong target.
- Encourages risk vs. reward mechanics, where players may choose to divert movement to collect a power-up instead of heading straight to the goal.

In [36]:
class PowerUp:
    """
    Represents a power-up object that grants a shield.
    """
    def __init__(self, canvas_size):
        self.canvas_size = canvas_size
        self.radius = 30  # Visual size
        self.position = np.array([
            random.randint(self.radius, canvas_size[1] - self.radius),
            random.randint(self.radius, canvas_size[0] - self.radius)
        ])
        self.type = "shield"
        self.duration = 300

    def draw(self, frame):
        """
        Draws the power-up as a filled yellow circle.
        """
        pos = self.position.astype(int)
        cv2.circle(frame, (pos[0], pos[1]), self.radius, (255, 255, 0), -1)

#### 4.6 ParticleEffect Class 
The ParticleEffect class is responsible for generating visual effects, primarily for explosions, impact feedback, or scoring events. It creates a dynamic particle system, where multiple particles move outward from a given position and gradually fade away, mimicking real-world effects such as smoke, sparks, or impact bursts.

Key Functionalities Include:

1. Initialization (__init__ method)
    - Takes a starting position and a number of particles (default: 50).
    - Generates individual particles with:
    - Random movement direction (using a random angle between 0 and 2π).
    - Random speed (between 2 and 8 pixels per frame).
    - Velocity components (vx, vy) derived from the speed and angle.
    - Random lifetime (20–40 frames), after which the particle disappears.
    - Stores the generated particles in a list, with each particle represented as a dictionary containing:
        - "pos" – Position (2D coordinate).
        - "vel" – Velocity (movement per frame).
        - "lifetime" – Number of frames before the particle disappears.

2. Updating and Rendering the Effect (update_and_draw method)

    - Updates each particle's position based on its velocity.
    - Reduces the lifetime of each particle.
    - Draws the particles using OpenCV (cv2.circle), coloring them based on the provided effect_color.
    - Removes expired particles (particles with lifetime <= 0).
    - Returns True if all particles have disappeared, signaling that the effect is complete.

##### How the ParticleEffect Class Contributes to Gameplay

- Provides instant feedback when an event occurs, such as:
    - Successful target hit (green particle burst).
    - Collision with an obstacle (red explosion effect).
    - Collecting a power-up (golden sparkle effect).
- Enhances visual appeal by making actions feel more dynamic and rewarding.
- Improves user engagement by adding a sense of motion and interaction.

[Table of content](#table-of-content)


In [37]:
class ParticleEffect:
    """
    Creates a particle effect (for explosions/feedback) when events occur.
    """
    def __init__(self, position, num_particles=50):
        self.particles = []
        for _ in range(num_particles):
            angle = random.uniform(0, 2 * np.pi)
            speed = random.uniform(2, 8)
            vx = speed * np.cos(angle)
            vy = speed * np.sin(angle)
            lifetime = random.randint(20, 40)
            self.particles.append({
                "pos": np.array(position, dtype=float),
                "vel": np.array([vx, vy], dtype=float),
                "lifetime": lifetime
            })

    def update_and_draw(self, frame, effect_color):
        """
        Updates particle positions and draws them. Removes expired particles.
        """
        for particle in self.particles:
            particle["pos"] += particle["vel"]
            particle["lifetime"] -= 1
            if particle["lifetime"] > 0:
                cv2.circle(frame, (int(particle["pos"][0]), int(particle["pos"][1])), 2, effect_color, -1)
        self.particles = [p for p in self.particles if p["lifetime"] > 0]
        return len(self.particles) == 0


### 4.7 Game Class (Main Controller)

The **Game** class serves as the main controller for the game. It manages the overall game flow, processes player input, updates game objects, detects collisions, handles scoring, renders visuals, and manages game-over conditions. This implementation integrates object tracking (via red object detection), physics (using a particle filter), and various game mechanics (targets, obstacles, power-ups) along with immersive feedback (sound and visual effects).

#### Key Functionalities

#### 4.7.1. Initialization (`__init__` method)

- **Audio System Setup:**
  - Loads and initializes sound effects for scoring, penalties, level-ups, power-ups, and combo bonuses using `pygame.mixer`.
  - Implements error handling in case audio files (e.g., `score.wav`, `penalty.mp3`, `levelup.mp3`, `Powerup.wav`, `combo.wav`) are missing.

- **Game Configuration:**
  - Defines the canvas size as **720x1280** (height x width) for display.
  - Sets parameters such as the number of particles (e.g., 300) for the particle filter and the motion noise level for simulating realistic movement.
  - Specifies properties for the ball and targets, including the supported shape types (circle, square, triangle, rectangle) and their corresponding dimensions.

- **Game State Variables:**
  - **Score:** Tracks the points earned by the player.
  - **Hearts (Lives):** The player starts with **15 hearts**.
  - **Level:** Game difficulty scales as the score increases (level-up every 5 points).
  - **Combo Mechanics:** Tracks consecutive successful target hits to apply a combo multiplier (with bonus sounds every three hits).
  - **Shield Status:** Indicates whether the player currently has an active shield power-up.

- **Object Initialization:**
  - **Ball:** Implements a particle filter to track its position based on input measurements from red object detection.
  - **Targets:** Creates target zones (with outlines) at designated corners, each having a unique shape.
  - **Obstacles:** Initializes moving obstacles that bounce off the screen edges and increase in speed as the level rises.
  - **Power-ups:** Randomly spawns power-ups that grant a temporary shield when collected.
  - **Particle Effects:** Generates visual effects for feedback on scoring and collisions.

- **Red Object Detection Setup:**
  - Uses OpenCV’s `VideoCapture` to access the webcam and flips the frame for intuitive control.
  - Applies a background subtractor along with HSV color thresholding to detect a moving red object.

#### 4.7.2 (Red object detection)
This method responsible for the handling of the detection of the red object held by the user. Within this method, we process each frame by first applying the background model to obtain a foreground mask. We then convert the frame to the HSV color space in order to define the color range for the object of interest; in this case, a red moving object. We then combine the red mask to the foreground mask in order to filter out only red moving objects. This is necessary because other moving objects could be detected in the foreground mask as well. Following this, we apply morphological operations such as erosion and dilation and the apply a Gaussian filter to filter out noise. For the resulting mask, we apply contour detection to find contours of red moving objects and the largest contour is assumed to be the region of interest. We then apply Convex Hull to find the farthest point which in this case is assumed to be the tip of the red object held by the user. 
Once this measurement is obtained, we pass the measurement to the update method of the ball which then proceeds with the prediction and update steps of the particle filter. 

#### 4.7.3. Main Game Loop (`run` method)

The `run` method drives the continuous game loop with the following steps:

- **Frame Processing & Input Handling:**
  - Captures frames from the webcam and flips them horizontally.
  - Processes each frame to detect the moving red object. The detection is performed using a combination of background subtraction and HSV filtering, and the detected red object’s tip serves as the measurement input.

- **Ball Tracking Update:**
  - The ball’s particle filter updates its state using the measurement from the red object detection.

- **Obstacle Movement Update:**
  - Obstacles update their positions each frame and bounce off the boundaries of the canvas.

- **Shield Timer Update:**
  - If a shield is active (from collecting a power-up), its duration counts down until it expires.

- **Collision & Scoring Checks:**
  - **Power-up Collection:**
    - If the ball collides with a power-up, a shield is activated (with associated sound and particle effects), and the power-up is removed from play.
  - **Obstacle Collisions:**
    - A collision between the ball and any obstacle causes the player to lose a heart (unless shielded), resets the combo counter, and triggers a red explosion effect along with a penalty sound.
  - **Target Zone Check:**
    - When the ball enters a target zone:
      - **Correct Shape:** Increases the score (with a combo multiplier) and triggers a green particle effect along with a scoring sound. A bonus combo sound is played every three consecutive hits.
      - **Wrong Shape:** Causes the loss of a heart (unless shielded) and triggers a red effect with a penalty sound.

- **Game Difficulty Scaling:**
  - The game levels up every 5 points. Leveling up increases the speed of the obstacles, making the game progressively more challenging.

- **Object Reset After Events:**
  - After any collision or scoring event, the game resets key objects:
    - A new ball is generated.
    - Targets and obstacles are re-randomized.

- **Particle Effects Update:**
  - Animates particle effects to provide visual feedback on impacts and scoring events.

- **Rendering:**
  - All game elements (ball, targets, obstacles, power-ups) and UI overlays (score, hearts, level, combo multiplier, shield status) are drawn on the frame using OpenCV.

- **Game Over Handling:**
  - If the player’s hearts drop to 0, a "GAME OVER" message is displayed before ending the game.

- **Exit Conditions:**
  - The game exits either when the player presses the 'Q' key or when the game-over condition is met.

#### 4.7.4. Cleanup and Resource Management

After exiting the main game loop:

- The webcam capture is released with `cap.release()`.
- All OpenCV windows are closed using `cv2.destroyAllWindows()`.
- Pygame’s mixer is shut down with `pygame.mixer.quit()` to free audio resources.

#### How the Game Class Contributes to Gameplay

- **Integrated Game Mechanics:** Combines obstacle avoidance, target matching, power-ups, and scoring to create a dynamic and engaging experience.
- **Interactive Control:** Utilizes red object detection to enable intuitive, gesture-based control of the ball.
- **Progressive Difficulty:** The game scales in challenge as the score increases by speeding up obstacles.
- **Immersive Feedback:** Enhances gameplay through synchronized sound effects and visual particle effects that react to in-game events.

[Click here to return to the table of content](#table-of-content)


In [38]:
class Game:
    """
    Main game controller class that manages initialization, input processing (detecting red controller held by user),
    game object updates, collision detection, scoring, and rendering.
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(message)s',
        handlers=[
            logging.FileHandler("game.log"),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    def __init__(self):
        pygame.mixer.init()
        try:
            self.score_sound = pygame.mixer.Sound("score.wav")
            self.penalty_sound = pygame.mixer.Sound("penalty.mp3")
        except Exception as e:
            self.logger.error("Could not load score/penalty sounds: %s", e)
            self.score_sound = None
            self.penalty_sound = None

        try:
            self.levelup_sound = pygame.mixer.Sound("levelup.mp3")
        except Exception as e:
            self.logger.error("Could not load levelup.wav: %s", e)
            self.levelup_sound = None

        try:
            self.powerup_sound = pygame.mixer.Sound("Powerup.wav")
        except Exception as e:
            self.logger.error("Could not load powerup.wav: %s", e)
            self.powerup_sound = None

        try:
            self.combo_sound = pygame.mixer.Sound("combo.wav")
        except Exception as e:
            self.logger.error("Could not load combo.wav: %s", e)
            self.combo_sound = None

        # Game configuration
        self.canvas_size = (720, 1280, 3)  # (height, width, channels)
        self.num_particles = 300
        self.motion_noise = 10
        self.shapes = ["circle", "square", "triangle", "rectangle"]
        self.ball_params = {
            "circle": {"radius": 50},
            "square": {"side": 60},
            "rectangle": {"width": 80, "height": 50},
            "triangle": {"side": 60}
        }
        self.target_params = { 
            "circle": {"radius": 70},
            "square": {"side": 100},
            "rectangle": {"width": 120, "height": 80},
            "triangle": {"side": 100}
        }

        self.score = 0
        self.hearts = 15
        self.last_heart_threshold = 0
        self.level = 1
        self.combo_counter = 0
        self.combo_multiplier = 1
        self.shield_active = False
        self.shield_timer = 0
        self.frame_counter = 0

        self.ball = Ball(self.canvas_size, self.ball_params, self.shapes, self.num_particles)
        self.targets = Target.create_targets(self.canvas_size, self.target_params, self.shapes)
        self.obstacles = [Obstacle(self.canvas_size, self.level) for _ in range(3)]
        self.powerups = []
        self.effects = []
        self.effect_color = None

        # Set up video capture
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.canvas_size[1])
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.canvas_size[0])
        # Create a background subtractor to remove static red objects
        self.bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=16, detectShadows=True)

    def detect_red_object(self, frame):
        """
        Detects a moving red object in the frame by first applying background subtraction
        to filter out static red regions and then detecting the largest red contour.
        Returns the measurement point (with added noise) or None if not found.
        """
        # Apply background subtraction to obtain the foreground mask
        fg_mask = self.bg_subtractor.apply(frame)
        # Convert the frame to HSV
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # Define red color ranges in HSV
        lower_red1 = np.array([0, 120, 70])
        upper_red1 = np.array([10, 255, 255])
        mask1 = cv2.inRange(hsv_frame, lower_red1, upper_red1)
        lower_red2 = np.array([170, 120, 70])
        upper_red2 = np.array([180, 255, 255])
        mask2 = cv2.inRange(hsv_frame, lower_red2, upper_red2)
        red_mask = mask1 + mask2

        # Combine the red mask with the foreground mask to filter out static objects
        combined_mask = cv2.bitwise_and(red_mask, red_mask, mask=fg_mask)
        # Clean up the mask
        combined_mask = cv2.erode(combined_mask, None, iterations=2)
        combined_mask = cv2.dilate(combined_mask, None, iterations=2)
        combined_mask = cv2.GaussianBlur(combined_mask, (7, 7), 0)

        # Find contours in the combined mask
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        measurement = None
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            # Compute the centroid of the contour
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                cX, cY = 0, 0
            # Use the convex hull to find the farthest point from the centroid (assumed tip)
            hull = cv2.convexHull(largest_contour)
            farthest_point = None
            max_distance = 0
            for point in hull:
                pt = point[0]
                distance = np.linalg.norm(np.array([cX, cY]) - pt)
                if distance > max_distance:
                    max_distance = distance
                    farthest_point = pt
            if farthest_point is not None:
                measurement = np.array([farthest_point[0], farthest_point[1]])
        return measurement

    def run(self):
        """
        Main game loop.
        """
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            self.frame_counter += 1

            # -----------------------------
            # Detect Red Object with Background Subtraction
            # -----------------------------
            measurement = self.detect_red_object(frame)
            # Uncomment to view the red mask (for debugging)
            # cv2.imshow("Red Object Mask", self.bg_subtractor.getBackgroundImage())

            # Update ball using the measurement from the red object detection
            self.ball.update(measurement, self.motion_noise)

            # Update obstacles
            for obs in self.obstacles:
                obs.update()

            # Update shield timer
            if self.shield_active:
                self.shield_timer -= 1
                if self.shield_timer <= 0:
                    self.shield_active = False

            ball_center = self.ball.get_position()

            # Check collisions with power-ups
            for powerup in self.powerups[:]:
                if np.linalg.norm(ball_center - powerup.position) < powerup.radius:
                    if powerup.type == "shield":
                        self.shield_active = True
                        self.shield_timer = 300
                        if self.powerup_sound:
                            pygame.mixer.Sound.play(self.powerup_sound)
                        self.effect_color = (0, 215, 255)
                    self.powerups.remove(powerup)
                    self.effects.append(ParticleEffect(powerup.position))

            # Occasionally spawn a power-up
            if self.frame_counter % 500 == 0:
                if random.random() < 0.5:
                    self.powerups.append(PowerUp(self.canvas_size))

            event_triggered = False

            # Check collision with obstacles
            collision = any(obs.collides_with(self.ball) for obs in self.obstacles)
            if collision:
                if self.shield_active:
                    self.shield_active = False
                else:
                    self.hearts -= 1
                    if self.penalty_sound:
                        pygame.mixer.Sound.play(self.penalty_sound)
                    self.effect_color = (0, 0, 255)
                self.combo_counter = 0
                self.combo_multiplier = 1
                self.effects.append(ParticleEffect(ball_center))
                event_triggered = True

            # Check if ball enters any target zone
            for target in self.targets:
                if target.point_inside(ball_center):
                    if self.ball.shape == target.shape:
                        self.effect_color = (0, 255, 0)
                        self.combo_counter += 1
                        self.combo_multiplier = 1 + (self.combo_counter // 3)
                        self.score += self.combo_multiplier
                        if self.score_sound:
                            pygame.mixer.Sound.play(self.score_sound)
                        if self.combo_counter % 3 == 0 and self.combo_sound:
                            pygame.mixer.Sound.play(self.combo_sound)
                        self.effects.append(ParticleEffect(target.position))
                    else:
                        if self.shield_active:
                            self.shield_active = False
                        else:
                            self.hearts -= 1
                        self.combo_counter = 0
                        self.combo_multiplier = 1
                        self.effect_color = (0, 0, 255)
                        if self.penalty_sound:
                            pygame.mixer.Sound.play(self.penalty_sound)
                        self.effects.append(ParticleEffect(target.position))
                    event_triggered = True
                    break

            # Level up every 5 points
            if self.score // 5 >= self.level:
                self.level = self.score // 5 + 1
                if self.levelup_sound:
                    pygame.mixer.Sound.play(self.levelup_sound)

            # Reset game objects if an event occurred
            if event_triggered:
                if self.score // 3 > self.last_heart_threshold:
                    self.hearts += 1
                    self.last_heart_threshold = self.score // 3
                self.ball = Ball(self.canvas_size, self.ball_params, self.shapes, self.num_particles)
                self.targets = Target.create_targets(self.canvas_size, self.target_params, self.shapes)
                self.obstacles = [Obstacle(self.canvas_size, self.level) for _ in range(3)]

            # Update and draw particle effects
            for effect in self.effects[:]:
                finished = effect.update_and_draw(frame, self.effect_color if self.effect_color else (255, 255, 255))
                if finished:
                    self.effects.remove(effect)

            # Draw game objects
            for target in self.targets:
                target.draw(frame)
            self.ball.draw(frame)
            for obs in self.obstacles:
                obs.draw(frame)
            for powerup in self.powerups:
                powerup.draw(frame)

            # Draw game UI
            cv2.putText(frame, f"Score: {self.score}", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 128, 0), 2)
            cv2.putText(frame, f"Hearts: {self.hearts}", (50, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, f"Level: {self.level}", (50, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 0, 75), 2)
            cv2.putText(frame, f"Combo: x{self.combo_multiplier}", (50, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 0, 75), 2)
            if self.shield_active:
                cv2.putText(frame, "Shield Active", (50, 250),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Check for game over
            if self.hearts <= 0:
                cv2.putText(frame, "GAME OVER", (self.canvas_size[1] // 2 - 150, self.canvas_size[0] // 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
                cv2.imshow("Shape Matching", frame)
                cv2.waitKey(3000)
                break

            cv2.imshow("Shape Matching", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        # Cleanup
        self.cap.release()
        cv2.destroyAllWindows()
        pygame.mixer.quit()

#### 5. Game Entry Point - Running the Game
Run the cell below (or execute the notebook) to launch the game. **Note:** The game opens a new window that accesses your webcam. Make sure your environment allows webcam access. Press **q** to quit the game.

Below we create a driver to initiate the program

[Click here to return to the table of content](#table-of-content)

In [42]:
# To run the game, simply run the notebook from this cell.
# If you are running in an environment that does not support a GUI or webcam,
# it is recommended to run this notebook on your local machine.

if __name__ == "__main__":
    game = Game()
    game.run()

## 6. Challenges and Possible Improvements

##### Challenges Faced:
- **Occlusion:** In real-world tracking scenarios, the target red object can be partially or fully occluded by other objects or the user's hand. Although our particle filter smooths out some noise, handling severe occlusions remains a challenge and may require more advanced tracking methods.
- **Lighting Conditions:** Variations in ambient lighting can adversely affect the performance of red object detection and video processing. Inconsistent lighting may lead to false detections or missed measurements, making robust pre-processing (e.g., adaptive histogram equalization) essential.
- **Predicting Non-Linearity of Object Motion:** The movement of the red object may be non-linear—sudden stops, abrupt direction changes, or erratic motions can occur. This unpredictability makes it challenging for the particle filter to accurately predict the next state based solely on previous frames. Hence, in the absence of measurement, the movement of the ball is purely random (although the program works quite well even in the event of measurement missing in a few couple of frames!)
- **Real-Time Performance:** Handling real-time video capture, processing particle updates, collision detection, and rendering all within a single loop can be demanding, especially on less powerful hardware. Optimizing performance without sacrificing accuracy is a key challenge. 

##### Lessons learned:
- We learned that the particle filter is indeed very good in capturing non-linearity in motion and hence, well-suited dealing with non-linear dynamic systems
- The background subtraction can be tricky if the background has movements (most especially if they have same colors as the object of interest)
- While traditional vision techniques such as the one employed in our implementation are fast, it becomes tedious to account for dynamic changes such as lighting conditions or occlusions; although the particle filter is great at handling these to some extent.
- We also came to the realization that traditional computer vision techniques are not suitable if multi-class detection and tracking is desired.
- In our experience, resampling the particles is a crucial step in enabling the particle filter to excel.
- Hand movements are not linear, and so any sudden stop or pause can hurdle the state estimation severely (particles will then turn to make random moves - as there is no measurement available in that case) although temporal stop or pauses does not seem to have any substatial negative effect on the algorithm. 
- The process of selecting the best parameters through experimentation and intuition can be a great pain (and we felt it 🙂)


#### Possible Improvements:
- **Enhanced Detection Parameters:** Instead of relying solely on a fixed HSV threshold, incorporating dynamic thresholding or multi-color detection could improve the robustness of the red object detection under varying lighting conditions.
- **Robust Object Tracking Methods:** Employing more advanced tracking techniques—such as deep learning–based object detectors or integrating optical flow—could mitigate issues related to occlusion and rapid motion.
- **Adaptive Particle Filtering:** Adjusting the number of particles dynamically based on the effective sample size or the complexity of the motion could improve the state estimation accuracy while maintaining computational efficiency.
- **Alternative State Estimation Algorithms:** While the particle filter works well for noisy measurements, exploring other state estimation methods (e.g., Unscented Kalman Filter) may offer improved performance in predicting complex, non-linear motions
- **Improved Collision Detection:** Fine-tuning the collision detection logic for obstacles and targets possibly by using more precise shape representations. This could reduce false collisions and improve game responsiveness.
- **User Interface Enhancements:** Enhancing the user interface with features such as a start menu, detailed instructions, real-time feedback for scoring and level-ups, or even visual indicators of the tracking accuracy could make the game more engaging and accessible.

Making these improvements would not only make the game more engaging and reliable but also enhance the applicability of our tracking algorithm in more complex, real-world scenarios. Thank you!

[Click here to go to table of content](#table-of-content)
